# wrchart Quickstart

Interactive financial charting with Polars and TradingView-style aesthetics.

In [ ]:
import sys
sys.path.insert(0, '..')

import wrchart as wrc
import polars as pl
import numpy as np

print(f'wrchart {wrc.__version__}')

## Generate Sample Data

In [ ]:
# Generate realistic OHLCV data
np.random.seed(42)
n = 252  # 1 year of trading days

returns = np.random.randn(n) * 0.02
prices = 100 * np.exp(np.cumsum(returns))

opens = np.roll(prices, 1)
opens[0] = prices[0]
highs = np.maximum(prices, opens) * (1 + np.abs(np.random.randn(n)) * 0.01)
lows = np.minimum(prices, opens) * (1 - np.abs(np.random.randn(n)) * 0.01)
volumes = np.random.randint(100000, 1000000, n)

df = pl.DataFrame({
    'time': list(range(n)),
    'open': opens,
    'high': highs,
    'low': lows,
    'close': prices,
    'volume': volumes,
})

df.head()

## Basic Candlestick Chart

In [ ]:
chart = wrc.Chart(width=900, height=500, title='OHLCV Candlestick Chart')
chart.add_candlestick(df)
chart.add_volume(df)
chart.show()

## Heikin-Ashi (Smoothed Candles)

In [ ]:
ha_df = wrc.to_heikin_ashi(df)

chart = wrc.Chart(width=900, height=400, title='Heikin-Ashi Chart')
chart.add_candlestick(ha_df)
chart.show()

## Renko Chart (Price-Based Bricks)

In [ ]:
renko_df = wrc.to_renko(df, brick_size=2.0)
print(f'Renko: {len(df)} bars -> {len(renko_df)} bricks')

chart = wrc.Chart(width=900, height=400, title='Renko Chart (brick=$2)')
chart.add_candlestick(renko_df)
chart.show()

## Chart with Moving Averages

In [ ]:
from wrchart.indicators import sma, ema

sma_20 = sma(df, period=20, value_col='close', output_col='value')
sma_50 = sma(df, period=50, value_col='close', output_col='value')

chart = wrc.Chart(width=900, height=500, title='Price with Moving Averages')
chart.add_candlestick(df)
chart.add_line(sma_20, color='#E53935', title='SMA 20')
chart.add_line(sma_50, color='#888888', title='SMA 50')
chart.show()

## High-Frequency Tick Data with LTTB Decimation

In [ ]:
# Simulate 100k ticks
n_ticks = 100_000
tick_df = pl.DataFrame({
    'time': list(range(n_ticks)),
    'value': 100 + np.cumsum(np.random.randn(n_ticks) * 0.01),
})

# Downsample to 2000 points for display
display_df = wrc.lttb_downsample(tick_df, target_points=2000)
print(f'Decimated: {n_ticks:,} ticks -> {len(display_df):,} points')

chart = wrc.Chart(width=900, height=400, title=f'Tick Data ({n_ticks:,} points)')
chart.add_line(display_df)
chart.show()

## Theme Comparison

In [ ]:
# Dark Theme
chart = wrc.Chart(width=900, height=400, theme=wrc.DarkTheme, title='Dark Theme')
chart.add_candlestick(df.head(100))
chart.show()

In [ ]:
# Light Theme (traditional green/red)
chart = wrc.Chart(width=900, height=400, theme=wrc.LightTheme, title='Light Theme')
chart.add_candlestick(df.head(100))
chart.show()